In [5]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization
import numpy as np
from PIL import Image

# Define your model architecture
def create_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Freeze the base model
    base_model.trainable = False
    
    # Add custom layers on top of the base model
    x = base_model.output
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Load your model weights
def load_my_model():
    model = create_model()
    model.load_weights('best_model.weights.h5')  # Load the weights
    return model

model = load_my_model()

# Image preprocessing function
def preprocess_image(image):
    image = image.resize((224, 224))  # Adjust size as per your model
    image = image.convert('RGB')  # Ensure image has 3 color channels
    image = np.array(image) / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Prediction function
def predict(image_path):
    image = Image.open(image_path)
    preprocessed_image = preprocess_image(image)
    prediction = model.predict(preprocessed_image)
    predicted_class = (prediction > 0.5).astype("int32")  # Assuming binary classification
    return "Right" if predicted_class[0][0] == 1 else "Wrong"

# Example usage
image_path = './Analog Section/train/class 0/fault_3.png'
result = predict(image_path)
print(f"The image is {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
The image is Right
